<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeClassificacaoDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Classificação

Import e instalação das bibliotecas necessárias

In [77]:
#Install e downloads
!pip install emoji
!pip install pandas
!pip install nltk
#nltk.download('all-nltk') #Demora um pouco

In [78]:
#Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [79]:
# Padrão RegEx
PADRAO_PALAVRAS_PT = r'[a-zA-Zà-úÀ-Ú0-9]+'

########## PATHS DE ARQUIVOS ##########

# Base após pré-processamento COM stopwords (Base 1)
PATH_BASE1_COM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base1_Com_StopWords.csv"  

# Base após pré-processamento SEM stopwords (Base 2)
PATH_BASE2_SEM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base2_Sem_StopWords.csv"

# Amostra Base 1
PATH_TESTE_BASE1 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base1_Com_StopWords.csv"  

# Amostra Base 2
PATH_TESTE_BASE2 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base2_Sem_StopWords.csv"

# Lista de Hates
PATH_LIST_HATE_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/odio_list.csv"

# Lista de Palavrões
PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/badword_list.csv"

In [80]:
#Carrega a base de dados com stopWords
base1 = pd.read_csv(PATH_BASE1_COM_STOPWORDS, index_col=0)
#base1 = pd.read_csv(PATH_TESTE_BASE1, index_col=0)
base1.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN,retwet eu queria compartilhar mas antes eu gos...
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil",gostei da pulseira e da educação dele nossaaa ...
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil",esses não irão ao encontro e beijing
5,1269998146601529344,Mon Jun 08 14:22:08 +0000 2020,@allantercalivre e eu me questionando: “quem ...,[],"Mossoró, Brasil",e eu me questionando quem é esse blogueiro sem...
6,1269997249746087939,Mon Jun 08 14:18:34 +0000 2020,@allantercalivre @marciolabre @jairbolsonaro R...,[],"Uberlândia, Brasil",renan caloteiro desonesto vejam link abaixo


In [81]:
#Carrega a base de dados sem stopWords
base2= pd.read_csv(PATH_BASE2_SEM_STOPWORDS, index_col=0)
#base2= pd.read_csv(PATH_TESTE_BASE2, index_col=0)
base2.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,text_cleanStopWords
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN,retwet eu queria compartilhar mas antes eu gos...,retwet queria compartilhar antes gostaria sabe...
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil",gostei da pulseira e da educação dele nossaaa ...,gostei pulseira educação nossaaa muita elegânc...
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil",esses não irão ao encontro e beijing,irão encontro beijing
5,1269998146601529344,Mon Jun 08 14:22:08 +0000 2020,@allantercalivre e eu me questionando: “quem ...,[],"Mossoró, Brasil",e eu me questionando quem é esse blogueiro sem...,questionando blogueiro blog juntos
6,1269997249746087939,Mon Jun 08 14:18:34 +0000 2020,@allantercalivre @marciolabre @jairbolsonaro R...,[],"Uberlândia, Brasil",renan caloteiro desonesto vejam link abaixo,renan caloteiro desonesto vejam link abaixo


In [82]:
base1.shape

(63380, 6)

In [83]:
base2.shape

(63380, 7)

Classificação Hate, Hate+ e outros

In [84]:
# Carrega dicionário hate
dict_hate = pd.read_csv(PATH_LIST_HATE_EXPRESSIONS_PT_BR)
dict_hate.head(5)

,hatelist
0,aberração
1,abestado
2,abestalhado
3,aborto
4,acabouamamata


In [85]:
# Carrega dicionário badword
dict_badword = pd.read_csv(PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR)
dict_badword.head(5)

,badword_list
0,babaca
1,boceta
2,boquete
3,bosseta
4,buceta


In [86]:
# Definição da função de classificação

def classifica(tweet, dicionario, LABEL1, LABEL2):
  # Tokeniza o tweet  
  tokenizer = RegexpTokenizer(PADRAO_PALAVRAS_PT)
  tokens = tokenizer.tokenize(tweet)
  
  chave = dicionario.keys()[0]

  lista = list(dicionario[chave])
  lista_expressoes = []

  # Gera lista de expressões
  for termo in lista:
    termo_tokens = tokenizer.tokenize(termo)
    if(len(termo_tokens)>1): #expressoes
      lista_expressoes.append(termo)

  # Verifica se o tweet possui ao menos uma PALAVRA de hate
  for token in tokens:
    if token in lista:
      return LABEL1

  # Verifica se o tweet possui ao menos uma EXPRESSÃO de hate
  for palavra in lista_expressoes:
    result = re.subn(r'^' + palavra + '\s', palavra + ' ', tweet) # Substitui no início
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '$',' '+ palavra, tweet) # Substitui no final
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '\s', ' '+palavra+ ' ', tweet) #Substitui no meio
    if(result[1]):
      break

  if(result[1]):
    return LABEL1
  else:
    return LABEL2

In [87]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 1
base1['classificacao_hate'] = base1['text_limpo'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base1.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,classificacao_hate
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil",gostei da pulseira e da educação dele nossaaa ...,not_hate
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil",esses não irão ao encontro e beijing,not_hate
5,1269998146601529344,Mon Jun 08 14:22:08 +0000 2020,@allantercalivre e eu me questionando: “quem ...,[],"Mossoró, Brasil",e eu me questionando quem é esse blogueiro sem...,not_hate
6,1269997249746087939,Mon Jun 08 14:18:34 +0000 2020,@allantercalivre @marciolabre @jairbolsonaro R...,[],"Uberlândia, Brasil",renan caloteiro desonesto vejam link abaixo,not_hate


In [88]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 2
base2['classificacao_hate'] = base2['text_limpo'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base2.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,text_cleanStopWords,classificacao_hate
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN,retwet eu queria compartilhar mas antes eu gos...,retwet queria compartilhar antes gostaria sabe...,not_hate
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil",gostei da pulseira e da educação dele nossaaa ...,gostei pulseira educação nossaaa muita elegânc...,not_hate
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil",esses não irão ao encontro e beijing,irão encontro beijing,not_hate
5,1269998146601529344,Mon Jun 08 14:22:08 +0000 2020,@allantercalivre e eu me questionando: “quem ...,[],"Mossoró, Brasil",e eu me questionando quem é esse blogueiro sem...,questionando blogueiro blog juntos,not_hate
6,1269997249746087939,Mon Jun 08 14:18:34 +0000 2020,@allantercalivre @marciolabre @jairbolsonaro R...,[],"Uberlândia, Brasil",renan caloteiro desonesto vejam link abaixo,renan caloteiro desonesto vejam link abaixo,not_hate


In [89]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 1
base1['classificacao_badword'] = base1['text_limpo'].apply(lambda x: classifica(x,dict_badword,"badword","not_badword"))
base1.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,classificacao_hate,classificacao_badword
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN,retwet eu queria compartilhar mas antes eu gos...,not_hate,not_badword
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil",gostei da pulseira e da educação dele nossaaa ...,not_hate,not_badword
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil",esses não irão ao encontro e beijing,not_hate,not_badword
5,1269998146601529344,Mon Jun 08 14:22:08 +0000 2020,@allantercalivre e eu me questionando: “quem ...,[],"Mossoró, Brasil",e eu me questionando quem é esse blogueiro sem...,not_hate,not_badword
6,1269997249746087939,Mon Jun 08 14:18:34 +0000 2020,@allantercalivre @marciolabre @jairbolsonaro R...,[],"Uberlândia, Brasil",renan caloteiro desonesto vejam link abaixo,not_hate,not_badword


In [90]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 1
base2['classificacao_badword'] = base2['text_limpo'].apply(lambda x: classifica(x,dict_badword,"badword","not_badword"))
base2.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,text_cleanStopWords,classificacao_hate,classificacao_badword
1,1270005650945343489,Mon Jun 08 14:51:57 +0000 2020,RT @franciscoedi11: @allantercalivre Eu queria...,[],NaN,retwet eu queria compartilhar mas antes eu gos...,retwet queria compartilhar antes gostaria sabe...,not_hate,not_badword
2,1270002784742539264,Mon Jun 08 14:40:34 +0000 2020,@allantercalivre @EdnaBraga Gostei da pulseira...,[],"Sao Paulo, Brazil",gostei da pulseira e da educação dele nossaaa ...,gostei pulseira educação nossaaa muita elegânc...,not_hate,not_badword
3,1270001502124417025,Mon Jun 08 14:35:28 +0000 2020,@allantercalivre @FBI Esses não irão ao encont...,[],"Recife, Brazil",esses não irão ao encontro e beijing,irão encontro beijing,not_hate,not_badword
5,1269998146601529344,Mon Jun 08 14:22:08 +0000 2020,@allantercalivre e eu me questionando: “quem ...,[],"Mossoró, Brasil",e eu me questionando quem é esse blogueiro sem...,questionando blogueiro blog juntos,not_hate,not_badword
6,1269997249746087939,Mon Jun 08 14:18:34 +0000 2020,@allantercalivre @marciolabre @jairbolsonaro R...,[],"Uberlândia, Brasil",renan caloteiro desonesto vejam link abaixo,renan caloteiro desonesto vejam link abaixo,not_hate,not_badword


In [91]:
# ELIMINAR OS TWEETS QUE CONTEM BADWORDS E SÃO NOT_HATE : drop (badword and not_hate)

baseDrop1 = base1[ (base1['classificacao_badword']=='badword') & (base1['classificacao_hate']=='not_hate')].index 
base1.drop(baseDrop1, inplace=True)

baseDrop2= base2[ (base2['classificacao_badword']=='badword') & (base2['classificacao_hate']=='not_hate')].index 
base2.drop(baseDrop2, inplace=True)

In [92]:
# Salva a base1 e base2
base1.to_csv("/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv") 
base2.to_csv("/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv")